# Tutorial: Pseudobulk eQTL Analysis with `cellink`

This tutorial demonstrates how to perform pseudobulk eQTL analysis using the `cellink` package, which integrates single-cell transcriptomics with genotype data. We focus on one cell type and test for cis-eQTLs across the genome, illustrating key steps such as pseudobulk aggregation, dimensionality reduction, filtering, and linear modeling.

This notebook assumes familiarity with single-cell data processing and basic statistical genetics concepts (e.g., eQTLs, PCA). If you're new to `cellink`, we recommend reviewing the [documentation page](#) for a conceptual overview of the package.

## Environment Setup

We begin by importing necessary libraries and defining key parameters for our analysis. `cellink` provides utilities that extend `AnnData` to handle both donor-level genotype data and single-cell RNA-seq data efficiently.

In [1]:
import gc
from pathlib import Path
import warnings

import anndata as ad
import scanpy as sc
import dask.array as da
import numpy as np
import pandas as pd
from statsmodels.stats.multitest import fdrcorrection
from tqdm.auto import tqdm

import cellink as cl
from cellink._core import DAnn, GAnn
from cellink.at.gwas import GWAS
from cellink.utils import column_normalize, gaussianize
from cellink.resources import get_onek1k

In [3]:
n_gpcs = 20
n_epcs = 15
batch_e_pcs_n_top_genes = 2000
chrom = 22
cis_window = 500_000
cell_type = "CD8 Naive"
celltype_key = "predicted.celltype.l2"
pb_gex_key = f"PB_{cell_type}"  # pseudobulk expression in dd.G.obsm[key_added]
original_donor_col = "donor_id"
min_percent_donors_expressed = 0.1
do_debug = False

## Load and Prepare Data

Here, we load a prepared dataset (`onek1k`) that includes genotype and expression information from human donors. We also extract gene annotations using Ensembl via `pybiomart`, which are essential for defining cis-windows during eQTL analysis.

In [ ]:
dd = get_onek1k(config_path='../../src/cellink/resources/config/onek1k.yaml')
dd

/Users/larnoldt/cellink_sample_data/onek1k/onek1k_cellxgene.h5ad already exists. Verifying checksum.
/Users/larnoldt/cellink_sample_data/onek1k/OneK1K.noGP.vcf.gz already exists. Verifying checksum.
/Users/larnoldt/cellink_sample_data/onek1k/OneK1K.noGP.vcf.gz.csi already exists. Verifying checksum.
/Users/larnoldt/cellink_sample_data/onek1k/gene_counts_Ensembl_105_phenotype_metadata.tsv.gz already exists. Verifying checksum.


/opt/miniconda3/envs/single_cell_base3/lib/python3.12/site-packages/pandas/core/internals/blocks.py:2588: RuntimeWarning: invalid value encountered in cast
  return self.values.astype(_dtype_obj)


╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │      DonorData(981 x n_donors=10,595,884, n_cells_per_donor=[333-3,511], donor_id = 'donor_id')  │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 981 ×         │ AnnData object with n_obs × n_vars = 1,248,980 ×   │
│ 10,595,884                                         │ 36,469                                             │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AC',         │     obs: 'orig.ident', 'nCount_RNA',               │
│ 'AC_Hemi', 'AC_Het', 'AC_Hom', 'AF', 'AN', 'ER2',  │ 'nFeature_RNA', 'percent.mt', 'donor_id',          │
│ 'ExcHet', 'HWE', 'IMPUTED', 'maf', 'NS', 'R2',     │ 'pool_number', 'predicted.celltype.l2',            │
│ 'TYPED', 'TYPED_ONLY', 'id', 'id_mask', 'length',  │ 'predicted.celltype.l2.score', 'age',              │
│ 'quality', 'pos_hg19', 'id_hg19'                   │ 'organism_ontology_term_id',                       │
│                                                    │ 'tissue_ontology_term_id',                         │
│                                                    │ 'assay_ontology_term_id',                          │
│                                                    │ 'disease_ontology_term_id',                        │
│                                                    │ 'cell_type_ontology_term_id',                      │
│                                                    │ 'self_reported_ethnicity_ontology_term_id',        │
│                                                    │ 'development_stage_ontology_term_id',              │
│                                                    │ 'sex_ontology_term_id', 'is_primary_data',         │
│                                                    │ 'suspension_type', 'tissue_type', 'cell_type',     │
│                                                    │ 'assay', 'disease', 'organism', 'sex', 'tissue',   │
│                                                    │ 'self_reported_ethnicity', 'development_stage',    │
│                                                    │ 'observation_joinid'                               │
│     uns: 'kinship'                                 │     var: 'vst.mean', 'vst.variance',               │
│                                                    │ 'vst.variance.expected',                           │
│                                                    │ 'vst.variance.standardized', 'vst.variable',       │
│                                                    │ 'feature_is_filtered', 'feature_name',             │
│                                                    │ 'feature_reference', 'feature_biotype',            │
│                                                    │ 'feature_length', 'feature_type'                   │
│     obsm: 'gPCs'                                   │     uns: 'cell_type_ontology_term_id_colors',      │
│                                                    │ 'citation', 'default_embedding',                   │
│                                                    │ 'schema_reference', 'schema_version', 'title'      │
│     varm: 'filter'                                 │     obsm: 'X_azimuth_spca', 'X_azimuth_umap',      │
│                                                    │ 'X_harmony', 'X_pca', 'X_umap'                     │
│                                                    │     varm: 'PCs'                                    │
└────────────────────────────────────────────────────┴─────────────────────────────────────────

In [ ]:
def _get_ensembl_gene_id_start_end_chr():
    from pybiomart import Server
    server = Server(host='http://www.ensembl.org')
    dataset = (server.marts['ENSEMBL_MART_ENSEMBL'].datasets['hsapiens_gene_ensembl'])
    ensembl_gene_id_start_end_chr = dataset.query(attributes=['ensembl_gene_id', 'start_position', 'end_position', 'chromosome_name'])
    ensembl_gene_id_start_end_chr = ensembl_gene_id_start_end_chr.set_index("Gene stable ID")
    ensembl_gene_id_start_end_chr = ensembl_gene_id_start_end_chr.rename(columns={
        "Gene start (bp)": GAnn.start,
        "Gene end (bp)": GAnn.end,
        "Chromosome/scaffold name": GAnn.chrom,
    })
    return ensembl_gene_id_start_end_chr

In [ ]:
ensembl_gene_id_start_end_chr = _get_ensembl_gene_id_start_end_chr()
ensembl_gene_id_start_end_chr

,start,end,chrom
Gene stable ID,,,
ENSG00000210049,577,647,MT
ENSG00000211459,648,1601,MT
ENSG00000210077,1602,1670,MT
ENSG00000210082,1671,3229,MT
ENSG00000209082,3230,3304,MT
...,...,...,...
ENSG00000197989,28578536,28583132,1
ENSG00000229388,28643159,28648829,1
ENSG00000289291,28736044,28737670,1


In [ ]:
dd.C.var = dd.C.var.join(ensembl_gene_id_start_end_chr)
dd.C.obs[DAnn.donor] = dd.C.obs[original_donor_col]
dd.G.obsm["gPCs"] = dd.G.obsm["gPCs"][dd.G.obsm["gPCs"].columns[:n_gpcs]]

## Normalize and Filter Expression Data

We normalize single-cell expression using total-count normalization and log transformation. We then filter the dataset to only include cells of a specific type (`CD8 Naive`). This step ensures we're analyzing homogeneous populations, which increases power in eQTL detection.

In [ ]:
sc.pp.normalize_total(dd.C)
sc.pp.log1p(dd.C)
sc.pp.normalize_total(dd.C)

mdata = sc.get.aggregate(dd.C, by=DAnn.donor, func="mean")
mdata.X = mdata.layers.pop("mean")

sc.pp.highly_variable_genes(mdata, n_top_genes=batch_e_pcs_n_top_genes)
sc.tl.pca(mdata, n_comps=n_epcs)

dd.G.obsm["ePCs"] = mdata[dd.G.obs_names].obsm["X_pca"]

In [ ]:
dd = dd[..., dd.C.obs[celltype_key] == cell_type, :].copy()
dd

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │        DonorData(980 x n_donors=10,595,884, n_cells_per_donor=[1-583], donor_id = 'donor_id')    │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 980 ×         │ AnnData object with n_obs × n_vars = 52,538 ×      │
│ 10,595,884                                         │ 36,469                                             │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AC',         │     obs: 'orig.ident', 'nCount_RNA',               │
│ 'AC_Hemi', 'AC_Het', 'AC_Hom', 'AF', 'AN', 'ER2',  │ 'nFeature_RNA', 'percent.mt', 'donor_id',          │
│ 'ExcHet', 'HWE', 'IMPUTED', 'maf', 'NS', 'R2',     │ 'pool_number', 'predicted.celltype.l2',            │
│ 'TYPED', 'TYPED_ONLY', 'id', 'id_mask', 'length',  │ 'predicted.celltype.l2.score', 'age',              │
│ 'quality', 'pos_hg19', 'id_hg19'                   │ 'organism_ontology_term_id',                       │
│                                                    │ 'tissue_ontology_term_id',                         │
│                                                    │ 'assay_ontology_term_id',                          │
│                                                    │ 'disease_ontology_term_id',                        │
│                                                    │ 'cell_type_ontology_term_id',                      │
│                                                    │ 'self_reported_ethnicity_ontology_term_id',        │
│                                                    │ 'development_stage_ontology_term_id',              │
│                                                    │ 'sex_ontology_term_id', 'is_primary_data',         │
│                                                    │ 'suspension_type', 'tissue_type', 'cell_type',     │
│                                                    │ 'assay', 'disease', 'organism', 'sex', 'tissue',   │
│                                                    │ 'self_reported_ethnicity', 'development_stage',    │
│                                                    │ 'observation_joinid'                               │
│     uns: 'kinship'                                 │     var: 'vst.mean', 'vst.variance',               │
│                                                    │ 'vst.variance.expected',                           │
│                                                    │ 'vst.variance.standardized', 'vst.variable',       │
│                                                    │ 'feature_is_filtered', 'feature_name',             │
│                                                    │ 'feature_reference', 'feature_biotype',            │
│                                                    │ 'feature_length', 'feature_type', 'start', 'end',  │
│                                                    │ 'chrom'                                            │
│     obsm: 'gPCs', 'ePCs'                           │     uns: 'cell_type_ontology_term_id_colors',      │
│                                                    │ 'citation', 'default_embedding',                   │
│                                                    │ 'schema_reference', 'schema_version', 'title',     │
│                                                    │ 'log1p'                                            │
│     varm: 'filter'                                 │     obsm: 'X_azimuth_spca', 'X_azimuth_umap',      │
│                                                    │ 'X_harmony', 'X_pca', 'X_umap'          

In [ ]:
gc.collect()

230

## Pseudobulk Aggregation and ePC Calculation

We compute pseudobulk expression profiles by averaging expression across all cells from the same donor. From these profiles, we identify highly variable genes and compute expression PCs (ePCs), which will be used as covariates in the regression model to control for expression heterogeneity.

In [9]:
dd.aggregate(key_added=pb_gex_key, sync_var=True, verbose=True)
dd.aggregate(obs=["sex", "age"], func="first", add_to_obs=True)
dd

[2025-02-10 19:00:16,399] INFO:cellink._core.donordata: Aggregated X to PB_CD8 NC
[2025-02-10 19:00:16,400] INFO:cellink._core.donordata: Observation found for 981 donors.


╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │         DonorData(981 x n_donors=143,083, n_cells_per_donor=[9-479], donor_id = 'donor_id')      │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 981 × 143,083 │ AnnData object with n_obs × n_vars = 133,482 ×     │
│                                                    │ 32,738                                             │
│     obs: 'sex', 'age'                              │     obs: 'orig.ident', 'nCount_RNA',               │
│                                                    │ 'nFeature_RNA', 'pool', 'individual',              │
│                                                    │ 'percent.mt', 'latent', 'nCount_SCT',              │
│                                                    │ 'nFeature_SCT', 'cell_type', 'cell_label', 'sex',  │
│                                                    │ 'age', 'donor_id'                                  │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AF', 'ER2',  │     var: 'GeneSymbol', 'features', 'start', 'end', │
│ 'maf', 'R2', 'contig', 'id', 'id_mask', 'quality'  │ 'chrom', 'strand', 'description', 'wikigene_name', │
│                                                    │ 'wikigene_id'                                      │
│     obsm: 'ePCs', 'PB_CD8 NC'                      │     uns: 'log1p'                                   │
│     varm: 'filter'                                 │                                                    │
└────────────────────────────────────────────────────┴────────────────────────────────────────────────────┘

## Filter Genes Based on Donor Expression Coverage

To ensure statistical robustness, we exclude genes that are not expressed in at least 10% of donors. This prevents unreliable associations driven by low expression or sparse data.

In [12]:
print(f"{pb_gex_key} shape:", dd.G.obsm[pb_gex_key].shape)
print("dd.shape:", dd.shape)

keep_genes = ((dd.G.obsm[pb_gex_key] > 0).mean(axis=0) >= min_percent_donors_expressed).values
dd = dd[..., keep_genes]
print("after filtering")
print(f"{pb_gex_key} shape:", dd.G.obsm[pb_gex_key].shape)
print("dd.shape:", dd.shape)

PB_CD8 NC shape: (981, 32738)
dd.shape: (981, 143083, 133482, 32738)
after filtering
PB_CD8 NC shape: (981, 14119)
dd.shape: (981, 143083, 133482, 14119)


## Prepare Covariate Matrix

We construct a covariate matrix including sex, age, genotype PCs (gPCs), and expression PCs (ePCs). These help control for confounding effects in the association analysis. Covariates are normalized before being used in the regression model.

In [13]:
F = np.concatenate(
    [
        np.ones((dd.shape[0], 1)),
        dd.G.obs[["sex"]].values - 1,
        dd.G.obs[["age"]].values,
        dd.G.obsm["gPCs"].values,
        dd.G.obsm["ePCs"],
    ],
    axis=1,
)
F[:, 2:] = column_normalize(F[:, 2:])

## Run cis-eQTL Mapping

We perform linear regression for each gene against all SNPs within a ±500kb cis-window. For each gene, we retain the top associated SNP and record statistics such as p-value, beta, and likelihood ratio test (LRT) values.

In [14]:
# alternative to dd[:, dd.G.var.chrom == str(chrom), :, dd.C.var.chrom == str(chrom)]
dd = dd.sel(G_var=dd.G.var.chrom == str(chrom), C_var=dd.C.var.chrom == str(chrom)).copy()
dd

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │         DonorData(981 x n_donors=143,083, n_cells_per_donor=[9-479], donor_id = 'donor_id')      │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 981 × 143,083 │ AnnData object with n_obs × n_vars = 133,482 × 360 │
│     obs: 'sex', 'age'                              │     obs: 'orig.ident', 'nCount_RNA',               │
│                                                    │ 'nFeature_RNA', 'pool', 'individual',              │
│                                                    │ 'percent.mt', 'latent', 'nCount_SCT',              │
│                                                    │ 'nFeature_SCT', 'cell_type', 'cell_label', 'sex',  │
│                                                    │ 'age', 'donor_id'                                  │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AF', 'ER2',  │     var: 'GeneSymbol', 'features', 'start', 'end', │
│ 'maf', 'R2', 'contig', 'id', 'id_mask', 'quality'  │ 'chrom', 'strand', 'description', 'wikigene_name', │
│                                                    │ 'wikigene_id'                                      │
│     obsm: 'ePCs', 'PB_CD8 NC', 'gPCs'              │     uns: 'log1p'                                   │
│     varm: 'filter'                                 │                                                    │
└────────────────────────────────────────────────────┴────────────────────────────────────────────────────┘

In [15]:
results = []
if isinstance(dd.G.X, da.Array | ad._core.views.DaskArrayView):
    if dd.G.is_view:
        dd._G = dd._G.copy()  # TODO: discuss with SWEs
    dd.G.X = dd.G.X.compute()

if do_debug:
    warnings.filterwarnings("ignore", category=RuntimeWarning)

for gene, row in tqdm(dd.C.var.iterrows(), total=dd.shape[3]):
    Y = gaussianize(dd.G.obsm[pb_gex_key][[gene]].values + 1e-5 * np.random.randn(dd.shape[0], 1))

    start = max(0, row.start - cis_window)
    end = row.end + cis_window
    _G = dd.G[:, (dd.G.var.pos < end)]
    _G = _G[:, (_G.var.pos > start)]
    _G = _G[:, (_G.X.std(0) != 0)]
    G = _G.X

    gwas = GWAS(Y, F)
    gwas.process(G)

    snp_idx = gwas.getPv().argmin()

    def _get_top_snp(arr, snp_idx=snp_idx):
        return arr.ravel()[snp_idx].item()

    rdict = {
        "snp": _G.var.iloc[snp_idx].name,
        "egene": gene,
        "n_cis_snps": G.shape[1],
        "pv": _get_top_snp(gwas.getPv()),
        "beta": _get_top_snp(gwas.getBetaSNP()),
        "betaste": _get_top_snp(gwas.getBetaSNPste()),
        "lrt": _get_top_snp(gwas.getLRT()),
    }
    results.append(rdict)

rdf = pd.DataFrame(results)
rdf

  0%|          | 0/360 [00:00<?, ?it/s]

,snp,egene,n_cis_snps,pv,beta,betaste,lrt
0,22_17476696_T_C,ENSG00000100181,1688,2.071223e-03,-0.349170,0.113373,9.485322
1,22_17739317_G_A,ENSG00000237438,2939,4.633734e-06,-0.937499,0.204662,20.983014
2,22_17732851_A_G,ENSG00000177663,3188,5.967313e-04,0.312597,0.091054,11.786146
3,22_17986900_G_A,ENSG00000069998,3378,3.983074e-04,1.301109,0.367420,12.540117
4,22_17206294_C_A,ENSG00000185837,3376,1.363376e-03,0.155577,0.048583,10.254690
...,...,...,...,...,...,...,...
355,22_51100218_C_T,ENSG00000205560,3415,1.832598e-03,-0.384827,0.123496,9.710082
356,22_51017082_T_A,ENSG00000100288,3337,2.352505e-04,-0.170751,0.046427,13.526244
357,22_51023924_C_T,ENSG00000205559,3317,9.775330e-05,0.355925,0.091354,15.179609
358,22_51064416_T_C,ENSG00000100299,3068,6.481592e-13,-0.489005,0.068012,51.695355


## Collect and Adjust Results

We collect all results into a DataFrame and perform multiple testing correction. The Benjamini-Hochberg FDR (`qv`) is computed to identify statistically significant gene-SNP pairs.

We then count the number of genes with significant associations (FDR < 0.05).

In [16]:
rdf["pv_adj"] = np.clip(rdf["pv"] * rdf["n_cis_snps"], 0, 1)  # gene-wise Bonferroni
rdf["qv"] = fdrcorrection(rdf["pv_adj"])[1]

(rdf.qv < 0.05).sum()

46